### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.

In [25]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [6]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [27]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.Operation;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpReadFlags;
import com.aerospike.client.exp.Expression;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.exp.ListExp;

import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
Key key1 = new Key("test", "testset", "key1");
Key key2 = new Key("test", "testset", "key2");
Key key3 = new Key("test", "testset", "key3");
WritePolicy wPolicy = new WritePolicy();
Bin b0 = new Bin("b00", Value.get(123)); 
Bin b1 = new Bin("b01", Value.get("abc")); 



In [28]:
String command = "date +%s%N";
 
try {
    Process process = Runtime.getRuntime().exec(command);
 
    BufferedReader reader = new BufferedReader(
            new InputStreamReader(process.getInputStream()));
    String line;
    while ((line = reader.readLine()) != null) {
        System.out.println(line);
    }
 
    reader.close();
 
} catch (IOException e) {
    e.printStackTrace();
}
b1 = new Bin("b01", Value.get("KEY 1"));
client.put(wPolicy, key1, b0, b1);

System.out.println("Record: "+ client.get(null, key1)); //check record got inserted

1713460155803847652
Record: (gen:1),(exp:451588155),(bins:(b00:123),(b01:KEY 1))


In [29]:
String command = "date +%s%N";
 
try {
    Process process = Runtime.getRuntime().exec(command);
 
    BufferedReader reader = new BufferedReader(
            new InputStreamReader(process.getInputStream()));
    String line;
    while ((line = reader.readLine()) != null) {
        System.out.println(line);
    }
 
    reader.close();
 
} catch (IOException e) {
    e.printStackTrace();
}
b1 = new Bin("b01", Value.get("KEY 2"));
client.put(wPolicy, key2, b0, b1);

System.out.println("Record: "+ client.get(null, key2)); //check record got inserted

1713460159028548843
Record: (gen:1),(exp:451588159),(bins:(b00:123),(b01:KEY 2))


In [30]:
String command = "date +%s%N";
 
try {
    Process process = Runtime.getRuntime().exec(command);
 
    BufferedReader reader = new BufferedReader(
            new InputStreamReader(process.getInputStream()));
    String line;
    while ((line = reader.readLine()) != null) {
        System.out.println(line);
    }
 
    reader.close();
 
} catch (IOException e) {
    e.printStackTrace();
}
b1 = new Bin("b01", Value.get("KEY 3"));
client.put(wPolicy, key3, b0, b1);

System.out.println("Record: "+ client.get(null, key3)); //check record got inserted

1713460160242915186
Record: (gen:1),(exp:451588160),(bins:(b00:123),(b01:KEY 3))


In [31]:
String command = "date +%s%N";
 
try {
    Process process = Runtime.getRuntime().exec(command);
 
    BufferedReader reader = new BufferedReader(
            new InputStreamReader(process.getInputStream()));
    String line;
    while ((line = reader.readLine()) != null) {
        System.out.println(line);
    }
 
    reader.close();
 
} catch (IOException e) {
    e.printStackTrace();
}

1713460163182180368


In [36]:
long lowLUT = 1713460155803847652L;
long highLUT = 1713460160242915186L;

In [37]:
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.RecordSet;

//Using expressions to read record metadata in PI that has record LUT
Expression lutRangeExp = Exp.build(Exp.and( 
                                    Exp.lt(Exp.val(lowLUT), Exp.lastUpdate()),
                                    Exp.gt(Exp.val(highLUT), Exp.lastUpdate())
                                    )
                                 );


//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
QueryPolicy qp = new QueryPolicy();
qp.filterExp = lutRangeExp;
RecordSet rs = client.query(qp, stmt);

while(rs.next()){
    Record r = rs.getRecord();
    Key thisKey = rs.getKey();  
    System.out.println(r);
    System.out.println(thisKey);
}


(gen:1),(exp:451588159),(bins:(b00:123),(b01:KEY 2))
test:testset:null:c10b2c764e604c439039804ec947fa2b18520d1a
(gen:1),(exp:451588155),(bins:(b00:123),(b01:KEY 1))
test:testset:null:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1


# Cleanup

In [5]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"